In [80]:
# Import necessary packages
import torch
import numpy as np
from dataset import Chexpert_dataset
import os
from torchvision.models import densenet121, DenseNet121_Weights
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

In [81]:
# Enable GPU use
os.environ['CUDE_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

In [82]:
# Get model
sex_model = densenet121(weights = DenseNet121_Weights.DEFAULT)

# Fix last layer of densenet to reflect number of classes
input_num = sex_model.classifier.in_features
sex_model.classifier = nn.Linear(input_num,1)

# Load pre-trained model
sex_model.load_state_dict(
    torch.load(os.path.join('pretrained_sex_model', "model.pt"), map_location=device)
)
sex_model = sex_model.to(device)

In [83]:
# Set in evaluation mode
sex_model.eval()

# Load data
# Set parameters to load training dataset
path_to_csv = 'model_train.csv'

# Root dir (needs to be changed depending on if I am using remote or gaon)
root_dir_gaon = '/export/gaon1/data/bbharti1'

# Additional specifications
columns = ['Sex']
transform = transforms.Resize((320,320))
all_data = Chexpert_dataset(path_to_csv,root_dir_gaon,columns,transform = transform)

In [85]:
# Evaluate model
# indices = np.arange(0,7000)
# sub_data = Subset(all_data,indices)
dataloader = DataLoader(all_data, batch_size=16, shuffle=False)
running_val_acc = 0

In [86]:
for i, data in enumerate(tqdm(dataloader)):
    image, label, image_paths = data
    image = image.to(device)
    label = label.to(device)

    output = sex_model(image)
    output = torch.sigmoid(output)
    val_batch_acc = torch.sum((output >=0.5) == label)/len(label)
    running_val_acc += val_batch_acc

100%|██████████| 9552/9552 [11:06<00:00, 14.33it/s]


In [79]:
running_val_acc/len(dataloader)

tensor(0.9944, device='cuda:0')